In [1]:
# ------------------------------------------------------------------
# One-cell Colab: Summarization prompt using GPT-2 (model + tokenizer)
# ------------------------------------------------------------------
# Paste this entire cell into Google Colab and run.
# It will install dependencies (if needed), load GPT-2, and provide a summarize() function.

# Install packages (skip if already installed in your runtime)
!pip install -q transformers accelerate --upgrade
!pip install -q sentencepiece   # sometimes useful for tokenization (safe to have)

# Imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import textwrap
import IPython.display as disp

# Choose model - small / medium / large options
# WARNING: larger models need more RAM / GPU memory. "gpt2" is smallest (124M).
MODEL_NAME = "gpt2"          # change to "gpt2-medium" / "gpt2-large" if you have enough GPU memory

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load tokenizer & model
print(f"Loading tokenizer & model: {MODEL_NAME} ... (this may take a minute)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# GPT-2 has no pad token by default; set pad_token to eos_token to allow padding/truncation safely
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Summarize function using prompt engineering + constrained generation
def summarize(
    text,
    prompt_header="Summarize the following text in 2-4 sentences:",
    max_summary_tokens=120,
    num_beams=4,
    length_penalty=1.0,
    no_repeat_ngram_size=3,
    temperature=0.7,
    top_p=0.9,
    return_raw=False
):
    """
    Summarize `text` using GPT-2 by constructing a prompt and generating continuation.
    Returns the summary string.
    """
    # Prepare prompt - keep it short and explicit
    # We'll place the text under a "TEXT:" marker, then ask for "Summary:" so model tends to produce a concise summary.
    prompt = f"{prompt_header}\n\nTEXT:\n{text}\n\nSummary:"

    # Tokenize and prepare input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=1024).to(device)

    # Generate: we ask the model to continue from "Summary:"
    # We set max_new_tokens to control summary length (GPT-2 models use generate differently).
    # Using num_beams for better quality; adjust temperature/top_p to control creativity.
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_summary_tokens,
            num_beams=num_beams,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            temperature=temperature,
            top_p=top_p,
            early_stopping=True,
            pad_token_id=model.config.pad_token_id,
            eos_token_id=model.config.eos_token_id
        )
    # The generated sequence contains the prompt + continuation; decode only the newly generated tokens
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)

    # Extract continuation after the last occurrence of the prompt marker "Summary:" (robust extraction)
    marker = "Summary:"
    if marker in generated_text:
        summary = generated_text.split(marker, 1)[1].strip()
    else:
        # fallback: remove the prompt part by length
        summary = generated_text[len(prompt):].strip()

    # Post-process: trim to first 3 sentences for brevity (optional)
    # We'll split on sentence-ending punctuation. Keep up to 3 sentences if available.
    sentences = []
    for part in summary.replace("\n", " ").split(". "):
        s = part.strip()
        if len(s) == 0:
            continue
        # put the period back (except maybe last)
        if not s.endswith("."):
            s = s + "."
        sentences.append(s)
        if len(sentences) >= 3:
            break
    brief_summary = " ".join(sentences).strip()

    if return_raw:
        return {"prompt": prompt, "generated_full": generated_text, "summary": brief_summary}
    return brief_summary

# --- Interactive usage example ---
example_text = (
    "Researchers at the university published a new paper describing a technique to improve battery life "
    "by using a novel electrode material. The experiments show a 20% increase in charge cycles and "
    "reduced degradation at high temperatures. This advance could enable longer-lasting electric vehicles "
    "and consumer electronics. The authors also discuss scaling up production and cost implications."
)

print("\nExample input (truncated):")
print(textwrap.shorten(example_text, width=300))
print("\nGenerated summary (example):")
print(summarize(example_text))

# --- Simple text input prompt for Colab users ---
print("\n\n=== Interactive mode: Enter your text to summarize ===")
user_text = input("Paste article/text (or press Enter to skip interactive input):\n")
if user_text.strip():
    print("\nSummary:\n", summarize(user_text))
else:
    print("No input provided. You can call summarize(your_text) in this cell after running.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 16.1 MB/s eta 0:00:00
Device: cpu
Loading tokenizer & model: gpt2 ... (this may take a minute)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Example input (truncated):
Researchers at the university published a new paper describing a technique to improve battery life by using a novel electrode material. The experiments show a 20% increase in charge cycles and reduced degradation at high temperatures. This advance could enable longer-lasting electric vehicles [...]

Generated summary (example):
The researchers found that the new electrode material could be used to improve the battery life of lithium-ion batteries, which are used in many consumer electronics products, such as cell phones, smartwatches, and smart TVs. The research was published in the journal Nature Communications. The paper was co-authored by researchers from the University of California, San Diego, and the National Institute of Standards and Technology (NIST).


=== Interactive mode: Enter your text to summarize ===
Paste article/text (or press Enter to skip interactive input):
Artificial intelligence (AI) has rapidly transformed industries across the world.